In [2]:
import pandas as pd
from src.mp_functions import knockout_FBA_w_tasks
from src.FBA_scripts.met_task_functions import constrain_model, read_tasks

"""Code to produce task lists with additional tasks for each tissue. At the bottom is code ro produce clean outputs."""



'Code to produce task lists with additional tasks for each tissue.'

In [2]:
#task_list_fix, removes should fail tasks
file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_orig.tsv'

tasks_df = pd.read_table(file_path, index_col=0)
print(tasks_df.shape[0])
tasks_df.drop(29, inplace=True)
tasks_df = tasks_df[tasks_df['shouldFail'].map(lambda x: x != 1)]
tasks_df.drop(['shouldFail', 'printFluxes', 'comments', 'changed', 'LBrxn', 'UBrxn'], axis=1, inplace=True) #  'LBequ_2', 'LBequ_3',  'UBequ_2', 'UBequ_3'
tasks_df.rename({'LBequ_1': 'LBequ', 'UBequ_1': 'UBequ'}, inplace=True, axis=1)
tasks_df.reset_index(inplace=True, drop=True)
print(tasks_df.shape[0])
tasks_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_new.tsv', sep='\t')


257
240


In [14]:
# Removes tasks already listed in essential tasks
file_path_1 = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv'
tasks_df_1 = pd.read_table(file_path_1, index_col=0)

file_path_2 = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_new.tsv'
tasks_df_2 = pd.read_table(file_path_2, index_col=0)

description_list = tasks_df_1['description'].tolist()

print(tasks_df_2.shape[0])
tasks_df_2 = tasks_df_2[tasks_df_2['description'].map(lambda x: not x in description_list)]
print(tasks_df_2.shape[0])
tasks_df_2.reset_index(inplace=True, drop=True)
tasks_df_2.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_minus_ess_new.tsv', sep='\t')

240
222


In [11]:
# Runs FBA to check tasks
file_path_tasks ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_minus_ess_new.tsv'
tissues = ['muscle']#, 'brain', 'pancreas', 'adrenal_gland', 'muscle',]

for tissue in tissues:
    model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)
    model_list = constrain_model(model_file_path, ALLMETSIN_IN=False)


    # if a metabolite is not found, it will skip the task
    # Any task where a metabolite is missing for a tissue needs to have its number added to 'ts'.
    # This way a 0 can be added for that task.
    # Also set missing to true
    # Tissues with missing tasks will need to be rerun with the tissue list changed.
    print('-----------------------')
    print('Checking tasks for ' + tissue + ':')
    tasks = read_tasks(file_path_tasks, model_list)

    # If there is only one tissue, a break can be added to add the missing task
##%%
    res = pd.DataFrame([[tissue, [], 0, []]], columns=['sample_ids', 'gene_ids', 'solution', 'tasks_results'])
    res = knockout_FBA_w_tasks(tasks, model_list, res)

    task_results = res.iat[0, 3]

    #Add task numbers in ts for those with missing metabolites
    ts = [219]
    for t in ts:
        task_results = task_results[:t] + [0] + task_results[t:]

    tasks_df = pd.read_table(file_path_tasks, index_col=0)
    tasks_df['REF'] = task_results
    tasks_df = tasks_df[tasks_df['REF'] == 1].drop(['REF', 'LBequ_2', 'LBequ_3', 'UBequ_2', 'UBequ_3'], axis=1)
    tasks_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/tissue_full_tasks/full_tasks_minus_ess_{0}.tsv'.format(tissue), sep='\t')
    print('Number of tasks for this tissue: ' + str(tasks_df.shape[0]))
    print('-----------------------')

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpe5lnosrk.lp
Reading time = 0.06 seconds
: 4714 rows, 13168 columns, 59336 nonzeros
-----------------------
Checking tasks for muscle:

Failed to find metabolite: bilirubin [Extracellular]. Skipping task: 219: Bilirubin conjugation (minimal substrates, physiological excretion)


In [3]:
# Make clean outputs from FBA results, makes it easy to see with task failed for which sample.

tissue_list = ['liver']#['pancreas', 'spleen', 'adipose_tissue', 'adrenal_gland', 'pituitary', 'thyroid', 'blood',
               #'heart', 'kidney', 'liver', 'muscle', 'nerve', 'lung']
tissue_list.sort()

for tissue in tissue_list:

    input_file = 'C:/Users/Sigve/Genome_Data/results/phewas_results/subcombs/start_stop_{0}_full_tasks_e.tsv'.format(tissue)
    output_file = 'C:/Users/Sigve/Genome_Data/results/phewas_results/subcombs/start_stop_{0}_full_tasks_e_c.tsv'.format(tissue)

    res = pd.read_table(input_file, index_col=0)
    res = res[res['tasks_results'] != "['ALL PASS']"]
    res['tasks_results'] = res['tasks_results'].apply(lambda x: [int(i) for i in x[1:-1].split(', ')])

    dict = {}
    for i, data in res.iterrows():
        dict[data['sample_ids']] = data['tasks_results']

    tasks_df_r = pd.DataFrame(dict)
    #print(tasks_df_r)

    file_path_t ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/tissue_full_tasks/full_tasks_minus_ess_{0}.tsv'.format(tissue)
    tasks_df_t = pd.read_table(file_path_t, index_col=0).reset_index()
    tasks_df_t['task_id'] = tasks_df_t['index']#
    tasks_df_t['_'*89 + 'description'] = tasks_df_t['description'].apply(lambda x:  '_'*(100-len(x)) + x)
    tasks_df_t = tasks_df_t[['task_id', '_'*89 + 'description']]

    tasks_df_t = tasks_df_t.merge(tasks_df_r, right_index=True, left_index=True)

    tasks_df_t.to_csv(path_or_buf=output_file, sep='\t')

